In [1]:
import string
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score

def strip_punct(s):
    sentences = []
    exclude = set(string.punctuation)
    exclude.add(']')
    exclude.add('[')
    exclude.add('\xad')
    exclude.add('\n')
    exclude.add('\n1')
    exclude.add('\n2')
    exclude.add('\n3')
    exclude.add('•')
    exclude.add('«')
    exclude.add('»')
    for i in range(len(s)):
        sentences.append(''.join(ch for ch in str(s[i]).lower() if ch not in exclude))
    return sentences

def vector_prep(l):
    l1 = []
    l2 = []

    for i in range(len(l)):
        l_str = ''
        l1.append(re.split(r'[/&]', str(l[i])))
        for x in range (len(l1[i])):
            l_str += l1[i][x] + ' '
        l2.append(' '.join(l_str.strip().split()))
    return l2

def feature(l1, l2, l3):
    feature = []
    for i in range(len(l1)):
        feature.append(l1[i].lower() + ' ' + l2[i].lower() + ' ' + l3[i].lower())
    return feature

def cros_val(model, x_train_array, price_train, n_run):
    print ('Cross-validation: {}'.format(cross_val_score(model, x_train_array, price_train, cv=n_run)))

/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
#1482535 rows x 8 columns
df = pd.read_csv('data/train.tsv', sep='\t')
df = df.sample(frac=1).reset_index(drop=True)

category_name_train = df['category_name'].iloc[14826:1482535].tolist()
name_train = df['name'].iloc[14826:1482535].tolist()
brand_train = df['brand_name'].iloc[14826:1482535].tolist()
price_train = df['price'].iloc[14826:1482535].tolist()

category_name_test = df['category_name'].iloc[0:14825].tolist()
name_test = df['name'].iloc[0:14825].tolist()
brand_test = df['brand_name'].iloc[0:14825].tolist()
price_test = df['price'].iloc[0:14825].tolist()

/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
category_l_train = vector_prep(category_name_train)
name_l_train = vector_prep(name_train)
brand_l_train = vector_prep(brand_train)

category_l_test = vector_prep(category_name_test)
name_l_test = vector_prep(name_test)
brand_l_test = vector_prep(brand_test)

feature_train = feature(category_l_train, name_l_train, brand_l_train)
feature_test = feature(category_l_test, name_l_test, brand_l_test)

In [5]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(feature_train)
#x_train_array = X.toarray()

In [6]:
model = LinearRegression()
model_l = model.fit(X, price_train)

In [8]:
#corpus_test = ['small cat harness and leash Other Pet Supplies Others']
newVec = CountVectorizer(vocabulary=vectorizer.vocabulary_)
X_test = newVec.fit_transform(feature_test)
prices_predicted = model_l.predict(X_test).tolist()

In [9]:
n_run = 5 #number of runs
cros_val(model_l, X, price_train, n_run)

Cross-validation: [ 0.38858986  0.40132084  0.39405723  0.38794164  0.37951911]


### SVR (support vector regression):

In [ ]:
from sklearn.svm import SVR

svr_poly  = SVR(kernel='poly', C=1e3, degree=2)
svr_model = svr_poly.fit(X, price_train)

In [ ]:
n_run = 5 #number of runs
cros_val(svr_model, X, price_train, n_run)

### Simple NN:

In [ ]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model